In [1]:
%pip install langchain
%pip install langchain-openai
%pip install faiss-cpu
%pip install requests
%pip install beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beh

# Getting and formating the Data from Website

## Fetching the website's content via the provided URL

In [2]:
import requests
from bs4 import BeautifulSoup
url=input("Enter the URL: ")
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

data = soup.text


Enter the URL: https://botpenguin.com/


In [3]:
data

"\n\n\n\n\n\n\n\n\nFree Chatbot maker | Chatbot for Website, WhatsApp | BotPenguin\n\n\n\n\n\n\n\n\n\n\n\nGot 10k+ Facebook followers? Get BotPenguin FREE for 6 monthsClaim close Why BotPenguin Integrations  arrow_right_alt Experience 60+ world-class integrations.Live Chat  arrow_right_alt Stay in the loop with your clientele 24*7!Unified Inbox  arrow_right_alt Serve your customers across all platforms.Key Features  arrow_right_alt Take your business to the next level with our awesome key features.Analytics  arrow_right_alt Speedtrack your growth with our top-of-the-line analytics tools!ComparisonsBotPenguin Vs. ManychatBotPenguin Vs. TidioBotPenguin vs Tawk toBotPenguin Vs. WatiBotPenguin Vs. InteraktBotPenguin vs AiSensyBotPenguin Vs. LandbotProduct What can it do?Who can use it?Where can it run?Marketing Automation  arrow_right_alt Make marketing a boon from the automation gods!WhatsApp Automation  arrow_right_alt Get that nifty automation for WhatsApp too!Customer Support  arrow_ri

## Splitting the data into chunks of documents

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
  chunk_size=1000, chunk_overlap=200, length_function=len, is_separator_regex=False,
)
data = splitter.create_documents([str(data)])
docs = splitter.split_documents(data)


In [5]:
docs

[Document(page_content='Free Chatbot maker | Chatbot for Website, WhatsApp | BotPenguin'),
 Document(page_content='Got 10k+ Facebook followers? Get BotPenguin FREE for 6 monthsClaim close Why BotPenguin Integrations  arrow_right_alt Experience 60+ world-class integrations.Live Chat  arrow_right_alt Stay in the loop with your clientele 24*7!Unified Inbox  arrow_right_alt Serve your customers across all platforms.Key Features  arrow_right_alt Take your business to the next level with our awesome key features.Analytics  arrow_right_alt Speedtrack your growth with our top-of-the-line analytics tools!ComparisonsBotPenguin Vs. ManychatBotPenguin Vs. TidioBotPenguin vs Tawk toBotPenguin Vs. WatiBotPenguin Vs. InteraktBotPenguin vs AiSensyBotPenguin Vs. LandbotProduct What can it do?Who can use it?Where can it run?Marketing Automation  arrow_right_alt Make marketing a boon from the automation gods!WhatsApp Automation  arrow_right_alt Get that nifty automation for WhatsApp too!Customer Support 

In [6]:
len(docs)

18

# Implementing the Chatbot

## Setting up the OPENAI_API_KEY for ChatGPT

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = input("YOUR_OPENAI_API_KEY: ")

## Storing the document data into VectorStore

In [8]:
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [9]:
VectorStore = FAISS.from_documents(docs, embeddings)

## Implementing RAG(Retrieval Augmented Generation)

In [10]:
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI

In [12]:
llm = OpenAI(temperature=0.3)

### Implementing the PromptTemplate for generating the required output

In [34]:
# please replace the name of BotPenguin with the name of your website from the template
from langchain.prompts import PromptTemplate
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.Answer the question as you are chatbot of BotPenguin. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


### Implementing the RAG chain

In [35]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=VectorStore.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [36]:
qa_chain({"query": "What is BotPenguin and what it do?"}, return_only_outputs=True)


{'result': ' BotPenguin is a chatbot maker that can be integrated on various platforms such as websites, WhatsApp, and more. It helps businesses with lead generation, customer service, sales, and marketing automation. It also offers multilingual support and can be used in various industries such as ecommerce, education, real estate, consulting, healthcare, and travel. Thanks for asking!'}

In [37]:
qa_chain({"query": "How many chatbot integration Bot Penguin have done?"})


{'query': 'How many chatbot integration Bot Penguin have done?',
 'result': ' BotPenguin has done 60+ chatbot integrations. Thanks for asking!'}

In [38]:
qa_chain({"query": "What are the key features of BotPenguin"})


{'query': 'What are the key features of BotPenguin',
 'result': '\nSome key features of BotPenguin include its ability to create chatbots for various platforms such as WhatsApp, Facebook Messenger, and Telegram, its integration with popular tools like ChatGPT and Google Calendar, its support for multiple languages, and its affordability. Thanks for asking!'}

In [39]:
qa_chain({"query": "  what are the messaging plateforms and tools it had done integrations "})


{'query': '  what are the messaging plateforms and tools it had done integrations ',
 'result': ' BotPenguin has integrations with over 60 native chatbot platforms, including CRMs, appointment scheduling tools, and analytics tools. Some of the specific platforms and tools it integrates with include LeadSquared, Zoho CRM, Plivo, GetResponse, Zendesk, Helpdesk, Groove CRM, ChatGPT, Bitrix 24, Hubspot, Google Sheets, GoHigh Level, Pabbly, Simply Book, Zapier, Drip CRM, Freshdesk, Jira, Cooper, and Google Calendar. Thanks for asking!'}

In [40]:
qa_chain({"query": "  what is your contact email?"})


{'query': '  what is your contact email?',
 'result': ' Our contact email is contact@botpenguin.com. Thanks for asking!'}

In [41]:
qa_chain({"query": "  who are you?"})

{'query': '  who are you?',
 'result': ' I am a chatbot created by BotPenguin to assist with any questions or inquiries you may have about our services. Thanks for asking!'}

In [42]:
def generate(query):
  if not query:
    return "Please enter the question"

  ans = qa_chain({"query":str(query)})
  return ans['result']

# Chatting with ChatBot

In [43]:
while True:
  query=input("Ask the Question or Enter exit-\n")
  if query=="exit":
    break
  print(generate(query))
  print()

Ask the Question or Enter exit-
Who are you?
 I am a chatbot created by BotPenguin, a company that specializes in creating chatbots for businesses. Thanks for asking!

Ask the Question or Enter exit-
exit
